# Cease to Flow Metric

This metric measures how regularly the river stops running. For the purposes of the evaluation, cease-to-flow conditions are defined to have occurred when the flow is less than 1 ML/d. Cease to flow events are predominately a Northern Basin consideration as it is much more intermittent than the South, where some permanently flowing rivers do not experience cease-to-flow periods. This metric uses the function: 


 
 $$ \frac{(365 - nCTF)}{log V_{inflow}}$$

where 
nCTF
 is the number of cease-to-flow days per year. This logarithmic approach allows a statistical comparison on a similar positive gradient to other metrics. Initially, several additional parameters were explored (days of cease-to-flow per year, maximum cease-to-flow period, average cease-to-flow period). However, after a number were initially investigated, the complexities of capturing multiple parameters using simple statistical tests led to the CtF analysis being rationalised to one indicator.
 
The model framework provides an imperfect representation of low flows, with some models rated as ‘poor’ for this part of the flow regime, therefore this metric is only assessed against an observed baseline.


## Inputs: 
[AWRA-L inflow data](https://data.gov.au/data/dataset/e65078cd-808d-4514-ab60-17e597b9a883/resource/7442a111-2894-4572-aa41-1f488bf06636)

[Gauges of interest for cease to flow](https://data.gov.au/data/dataset/7c44535b-4a6a-432d-acff-00ec578ce7b9/resource/e40371ed-ae53-4cf7-91e7-1f66bae31008)

## Outputs:
[Results](https://data.gov.au/data/dataset/hydrologic-indicator-results-for-the-basin-plan-evaluation-2020)

In [0]:
import pandas as pd
from datetime import datetime
import numpy as np
import scipy.stats
import warnings 
warnings.filterwarnings('ignore')

##Load catchment inflow data

Load daily inflow data for each catchment from Australian Landscape Water Balance Landscape model. 

The data is loaded in, cleaned, and grouped by the water year 

This dataset is stored in https://data.gov.au/data/dataset/e65078cd-808d-4514-ab60-17e597b9a883/resource/7442a111-2894-4572-aa41-1f488bf06636

Inflows are given by:  
 $$ Inflows = Runoff \times Surface Area$$

Where runoff from 1911 - 2018/19 was provided by Bureau of Meteorology’s (BoM) AWRA Modelling Team from the Australian Water Resources Assessment Landscape model (AWRA-L) version 6.0. 

Surface Area calculated from shapefile of catchments (available [here](https://services8.arcgis.com/5xxEi7I2m6ml97fE/arcgis/rest/services/BASIN_PLAN_REGIONS/FeatureServer))

In [0]:
RawBOMData = pd.read_csv('https://data.gov.au/data/dataset/e65078cd-808d-4514-ab60-17e597b9a883/resource/7442a111-2894-4572-aa41-1f488bf06636/download/catchmentinflows_modelledrunoffdata_awralv6.csv')
RawBOMData.head()

,Column1,Barwon-Darling,Border Rivers,Campaspe,Condamine-Balonne,Eastern Mt Lofty Ranges,Goulburn-Broken,Gwydir,Lachlan,Loddon,Lower Darling,Macquarie-Castlereagh,Moonie,Murray,Murrumbidgee,Namoi,Ovens,Paroo,Warrego,Wimmera-Avoca
0,1/01/1911,280.945743,889.116774,448.182780,464.969232,731.547092,5352.450565,842.971632,3708.877278,244.310367,361.576414,1989.390134,10.016151,32348.09683,12736.194310,1018.087050,7820.429640,197.437945,311.490847,318.456924
1,2/01/1911,245.939033,931.281696,354.872346,551.652614,561.807131,4426.229832,869.601585,2982.273356,217.188593,267.220044,1854.414908,7.661954,25171.03952,10177.408170,1024.492008,6337.477974,175.114516,365.595952,284.131808
2,3/01/1911,221.136955,996.579011,287.529469,656.670069,443.333880,3747.495391,1056.964622,2521.920740,197.458266,207.665376,1802.773964,6.639534,19788.21003,8278.185418,1030.521136,5224.703577,161.117734,380.395537,259.232940
3,4/01/1911,228.037716,906.414042,238.892884,565.381870,360.467282,3252.306652,1344.195169,2588.127451,183.177053,172.462025,1928.902206,5.369864,16204.23390,7446.809863,1009.258218,4470.305920,152.364875,332.566101,240.716578
4,5/01/1911,282.894836,780.040817,211.475708,494.168382,302.345780,3000.776435,1054.972176,3047.736346,185.881273,172.426760,2834.161997,4.337170,15689.32201,7848.837431,1056.232902,4109.854376,156.725326,300.506739,231.350913


In [0]:
def transformPipline(RawDataframe):
    """
    Single function to transform raw dataframe from blob into a pandas dataframe ready for analysis
    """

    # Turn Column1 into Date

    DailyRunoffDataframe = RawDataframe.rename({'Column1':'Date'}, axis =1 )

    # total up northen basin catchments

    NorthernBasinCatchments = [
        'Barwon-Darling',
        'Border Rivers',
        'Condamine-Balonne',
        'Gwydir',
        'Macquarie-Castlereagh',
        'Moonie',
        'Namoi',
        'Paroo',
        'Warrego',
        ]

    DailyRunoffDataframe['Northern Basin'] = \
        DailyRunoffDataframe.apply(lambda row: \
                                   row[NorthernBasinCatchments].sum(),
                                   axis=1)

    # total up southern basin catchments

    SouthernBasinCatchments = [
        'Campaspe',
        'Eastern Mt Lofty Ranges',
        'Goulburn-Broken',
        'Lachlan',
        'Loddon',
        'Lower Darling',
        'Murray',
        'Murrumbidgee',
        'Ovens',
        'Wimmera-Avoca',
        ]

    DailyRunoffDataframe['Southern Basin'] = \
        DailyRunoffDataframe.apply(lambda row: \
                                   row[SouthernBasinCatchments].sum(),
                                   axis=1)

    # total up all catchments

    AllCatchments = NorthernBasinCatchments + SouthernBasinCatchments

    DailyRunoffDataframe['Total MDB'] = \
        DailyRunoffDataframe.apply(lambda row: \
                                   row[AllCatchments].sum(), axis=1)

    # convert to a datetime data type

    DailyRunoffDataframe['Date'] = \
        pd.to_datetime(DailyRunoffDataframe['Date'], format='%d/%m/%Y')

    # drop Nulls

    DailyRunoffDataframe = DailyRunoffDataframe.dropna()

    return DailyRunoffDataframe

In [0]:
# Based on the month of a date, work out what water year that date belongs to


def waterYear(date):
    '''Takes in date,
  changes year to water year
  returns water year'''
    if date.month <= 6:  # for months Jan to Jun move them to the previous water year
        waterYear = date.year - 1
    else:

         # for months after Jun move them to this  water year

        waterYear = date.year
    return int(waterYear)

In [0]:
DailyRunoffDataframe = transformPipline(RawBOMData)

# apply water year function to populate the water year column

DailyRunoffDataframe['Water Year'] = \
    DailyRunoffDataframe.apply(lambda row: waterYear(row['Date']),
                               axis=1)

In [0]:
# summing annual inflow by water year

AnnualRunoffDataframe = DailyRunoffDataframe.groupby('Water Year',
        as_index=False).sum()

## Load gauges of interest for Cease to Flow metric
Gets gauges of interest and their associated gauge data.

Cease to Flow gauge information can be found: https://data.gov.au/data/dataset/7c44535b-4a6a-432d-acff-00ec578ce7b9/resource/e40371ed-ae53-4cf7-91e7-1f66bae31008

In [0]:
Data = pd.read_csv('https://data.gov.au/data/dataset/7c44535b-4a6a-432d-acff-00ec578ce7b9/resource/e40371ed-ae53-4cf7-91e7-1f66bae31008/download/observedflows_ceasetoflows.csv', header=None)
Data.head()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3357486605271491> in <module> 
 ----> 1 Data = pd . read_csv ( 'https://data.gov.au/data/dataset/7c44535b-4a6a-432d-acff-00ec578ce7b9/resource/e40371ed-ae53-4cf7-91e7-1f66bae31008/download/observedflows_ceasetoflows.csv' , header = None ) 
 2 Data . head ( ) 

 NameError : name 'pd' is not defined

## Transform gauge data 

Organising dataframe to get it ready for analysis:
- Putting gauge numbers as column headings
- stripping header information and using this data to filter to only the locations of interest

In [0]:
DataFrame = Data.loc[3:]
DataFrame.columns = map(str.strip,
                           Data.loc[2].astype(str).tolist())
DataFrame['Date'] = DataFrame['Date'].apply(pd.to_datetime, format='%d/%m/%Y')

DataFrame.set_index('Date')

DataFrame.drop(DataFrame.tail(64).index,inplace=True)

DataFrame.head()

,Date,422002,422006,416001,416052,418055,419049,419026,421011,421012,421023,424002,423001,417001,422003,422001,425003,425004,425008,425012,406202,405232,407202,403200,412045,425007,410130,415246,425018,422004,412005,407205
3,1905-07-04,3764.871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1905-07-05,3344.487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1905-07-06,2892.733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1905-07-07,2315.572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1905-07-08,1867.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
CatchmentGaugeMapping = Data.loc[0:2]
CatchmentGaugeMapping.columns = Data.loc[2].str.strip().tolist()
CatchmentGaugeMapping = CatchmentGaugeMapping.drop('Date', axis=1)
CatchmentGaugeMapping = CatchmentGaugeMapping.transpose()
CatchmentGaugeMapping.columns = ["catchment",'Name', 'Gauge']

CatchmentGaugeMapping.loc[CatchmentGaugeMapping.catchment
                          == 'Overall South ', 'catchment'] = \
    'SouthernBasin'
CatchmentGaugeMapping.loc[CatchmentGaugeMapping.catchment
                          == 'Overall MDBA System ', 'catchment'] = \
    'TotalMDB'

CatchmentGaugeMapping.head()

,catchment,Name,Gauge
422002,Northern Basin,BARWON RIVER AT BREWARRINA,422002
422006,Condamine-Balonne,CULGOA RIVER AT D/S COLLERINA (KENEBREE),422006
416001,Border Rivers,BARWON RIVER AT MUNGINDI,416001
416052,Border Rivers,GIL GIL CREEK AT GALLOWAY,416052
418055,Gwydir,MEHI RIVER AT NEAR COLLARENEBRI,418055


In [0]:
DataFrame['water year'] = DataFrame.apply(lambda row: \
        waterYear(row['Date']), axis=1)

In [0]:
# Filtering the data to only include observed flow data after the cap on diversions was introduced (1994):
DataFramefilter = DataFrame[DataFrame["water year"] >= 1994]
DataFramefilter.head()

,Date,422002,422006,416001,416052,418055,419049,419026,421011,421012,421023,424002,423001,417001,422003,422001,425003,425004,425008,425012,406202,405232,407202,403200,412045,425007,410130,415246,425018,422004,412005,407205,water year
32507,1994-07-01,17.867,0,0,0.906,1.494,0,80.126,0,44.998,26.912,0,0,0,0.246,69.452,0,26.042,39.582,313.926,14.83,2605.98,306.87,3461.28,206.824,274.778,2738.53,17.52,0,0.243,210.939,81.73,1994
32508,1994-07-02,20.154,0,0,0.906,1.494,0,82.007,0,48.237,24.037,0,0,0,0,62.974,0,25.825,38.133,313.844,21.05,2460.09,368.63,3074.08,203.616,271.392,2242.58,17.51,0,0.16,213.507,93.1,1994
32509,1994-07-03,24.082,0,0,0.906,1.494,0,81.093,0,32.535,22.149,0,0,0,0,61.635,0,25.609,38.113,314.913,26.62,2043.7,385.78,2711.05,201.035,263.965,1762.94,17.74,0,0.087,206.231,100.79,1994
32510,1994-07-04,40.006,0,0,1.035,1.494,0,80.623,0,51.946,19.988,0,0,0,0,63.067,0,25.393,35.759,312.013,25.43,1590.57,407.55,2471.59,198.387,264.636,1386.06,17.85,0,0.018,202.245,103.81,1994
32511,1994-07-05,45.113,0,0,1.165,1.247,0,80.281,0,41.166,19.158,0,0,0,0,65.249,0,25.178,34.323,311.891,22.1,1243.94,428.06,2309.82,197.967,264.485,820.453,17.8,0,0,187.653,105.18,1994


In [0]:
meltedDataFrame = pd.melt(DataFramefilter, id_vars=['Date',
                             'water year'], var_name='ID',
                             value_name='Outflow')
meltedDataFrame['Outflow'] = \
    pd.to_numeric(meltedDataFrame['Outflow'])

## Calculating the CtF days per water year

In [0]:
#counting CtF days per water year (where flow is less than 1 ML/day)
Ctfdataframe = meltedDataFrame[meltedDataFrame['Outflow']
                                  <= 1].groupby(['ID', 'water year'],
        as_index=False).count().drop(['Outflow'], axis=1)

In [0]:
# Filtering out sites with no signficant CtF events

CtfdataframeCount = Ctfdataframe.groupby('ID', as_index=False).count()

CtfdataframeFilltered = Ctfdataframe[Ctfdataframe['ID'
        ].isin(CtfdataframeCount[CtfdataframeCount['Date'] >= 10]['ID'
        ].unique().tolist())]

Ctfdataframepivot = CtfdataframeFilltered.pivot_table(index='ID',
        columns='water year', values='Date', fill_value=0, aggfunc='sum'
        ).unstack().to_frame().rename(columns={0: 'CTFdays'})
Ctfdataframepivot = Ctfdataframepivot.reset_index(drop=False)

Ctfdataframepivot.head()

,water year,ID,CTFdays
0,1994,407205,0
1,1994,415246,84
2,1994,416001,261
3,1994,416052,187
4,1994,417001,233


## Adding catchment inflow details

In [0]:
def gaugetocatchment(gaugeID):
  '''Takes in gauge,
  returns catchment it is in'''
  
  return CatchmentGaugeMapping.loc[gaugeID]["catchment"]

gaugetocatchment("407205")

Out[23]: 'Loddon'

In [0]:
Ctfdataframepivot['catchment'] = Ctfdataframepivot.apply(lambda x: \
        gaugetocatchment(x['ID']), axis=1)


meltedinflows = pd.melt(AnnualRunoffDataframe, id_vars=['Water Year'],
                        var_name='catchment', value_name='inflow')

meltedinflows.loc[meltedinflows.catchment == 'Total MDB', 'catchment'
                  ] = 'TotalMDB'

Ctfdataframemerged = Ctfdataframepivot.merge(meltedinflows,
        left_on=['catchment', 'water year'], right_on=['catchment',
        'Water Year'], how='left')
Ctfdataframemerged.head()

,water year,ID,CTFdays,catchment,Water Year,inflow
0,1994,407205,0,Loddon,1994,201571.194061
1,1994,415246,84,Wimmera-Avoca,1994,235127.620391
2,1994,416001,261,Border Rivers,1994,591665.697011
3,1994,416052,187,Border Rivers,1994,591665.697011
4,1994,417001,233,Moonie,1994,27874.971242


## Calculating the CtF Ratio
See command 1 for the CtF ratio formula

In [0]:
Ctfdataframemerged = Ctfdataframemerged.drop(['Water Year'], axis=1)
Ctfdataframemerged['Ratio'] = Ctfdataframemerged.apply(lambda row: (365
        - row['CTFdays']) / np.log(row['inflow']), axis=1)
Ctfdataframemerged.head()

,water year,ID,CTFdays,catchment,inflow,Ratio
0,1994,407205,0,Loddon,201571.194061,29.883990
1,1994,415246,84,Wimmera-Avoca,235127.620391,22.720136
2,1994,416001,261,Border Rivers,591665.697011,7.825022
3,1994,416052,187,Border Rivers,591665.697011,13.392827
4,1994,417001,233,Moonie,27874.971242,12.896312


## Compare pre and post Basin Plan
Compare the pre and post Basin Plan baseflow index using:
- Welsh's T-test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html)
- the KS two sample test (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ks_2samp.html)

In [0]:
def siteloopGauge(ResultsDataFrame, Gauge, quiet=True):
    '''Takes in dataframe with ctf data and date,
  filters the dataframe to pre and post basin plan periods,
  runs  Welsh's t test and ks two sample test on both periods,
  returns the results dataframe'''
    pre = np.array(ResultsDataFrame[(ResultsDataFrame['water year']
                   < 2012) & (ResultsDataFrame['ID'] == Gauge)]['Ratio'
                   ])
    post = np.array(ResultsDataFrame[(ResultsDataFrame['water year']
                    >= 2012) & (ResultsDataFrame['ID']
                    == Gauge)]['Ratio'])

    (ksStat, KsP) = scipy.stats.ks_2samp(pre, post)
    (tStat, tP) = scipy.stats.ttest_ind(pre, post, equal_var=False)

    Outcome = Significant(KsP, tStat, tP, alpha)

    if not quiet:
        print (Gauge, scipy.stats.ks_2samp(pre, post))
        print (Gauge, scipy.stats.ttest_ind(pre, post, equal_var=False))
    
    StepDataFrame = pd.DataFrame({
      "Catchment":[gaugetocatchment(Gauge)], 
      "ID":[str(Gauge)], 
      "Metric":["Cease to Flow"], 
      "Source":["Observed"],
      "Ks_2sampResult statistic":[ksStat], 
      "Ks_2sampResult pvalue":[KsP], 
      "Welch’s t-test statistic":[tStat], 
      "Welch’s t-test pvalue":[tP], 
      "Outcome":[Outcome]}) 
  
    return StepDataFrame


def Significant (Ksp, tStat, tP, alpha):
  '''Takes in results of statistical tests,
  compares the results of the two tests to an alpha value defined by the operator,
  returns the significance'''
  if ((Ksp < alpha) and (tStat <0) and (tP < alpha)):
    outcome = "Improved" 
  elif (tStat >0 and Ksp <alpha and tP < alpha):
    outcome = "Degraded" 
  elif (Ksp >alpha and tP > alpha):
    outcome = "Maintained" 
  elif (Ksp <alpha and tP > alpha):
    outcome = "Unsure - t-test failed" 
  else:
    outcome = "Unsure - ks-test failed"
  return outcome

##Selecting an Alpha 
With two tests with alphas set at 0.1, the probability of observing a false statistically significant results in both tests is 1%  

Typically, methods for dealing with multiple tests call for adjusting alpha in some way, however, these methods are designed for statistical investigations looking for a single significant result, ‘a discovery’. This is not the case in the application of two statistical tests looking for concurrent significant results.  

Setting alpha to 0.1 in both tests so that the chance of a false positive ‘increased’ or ‘decreased’ result is 1% is suitably rigorous and decidedly reasonable for the task at hand.

In [0]:
alpha = 0.1
  
StatsResultsGauge = pd.DataFrame(data=[],columns = [
  "Catchment", 
  "ID", 
  "Metric", 
  "Source", 
  "Ks_2sampResult statistic", 
  "Ks_2sampResult pvalue", 
  "Welch’s t-test statistic", 
  "Welch’s t-test pvalue", 
  "Outcome"
  ])  
  
for gauge in Ctfdataframemerged["ID"].unique():
  
  StepDataFrame = siteloopGauge(Ctfdataframemerged, gauge) 
  StatsResultsGauge = StatsResultsGauge.append(StepDataFrame)  
  
StatsResultsGauge = StatsResultsGauge.replace({'Northern Basin': 'Barwon-Darling'}, regex=True)
StatsResultsGauge

,Catchment,ID,Metric,Source,Ks_2sampResult statistic,Ks_2sampResult pvalue,Welch’s t-test statistic,Welch’s t-test pvalue,Outcome
0,Loddon,407205,Cease to Flow,Observed,0.801587,0.001119,-2.700750,0.014210,Improved
0,Wimmera-Avoca,415246,Cease to Flow,Observed,0.547619,0.060690,-2.573031,0.022404,Improved
0,Border Rivers,416001,Cease to Flow,Observed,0.285714,0.728287,-0.023813,0.981423,Maintained
0,Border Rivers,416052,Cease to Flow,Observed,0.444444,0.199883,1.380138,0.195271,Maintained
0,Moonie,417001,Cease to Flow,Observed,0.634921,0.018222,2.970140,0.012817,Degraded
0,Gwydir,418055,Cease to Flow,Observed,0.214286,0.951802,0.480799,0.643215,Maintained
0,Namoi,419026,Cease to Flow,Observed,0.373016,0.392099,0.835256,0.427976,Maintained
0,Namoi,419049,Cease to Flow,Observed,0.373016,0.392099,0.999648,0.341620,Maintained
0,Macquarie-Castlereagh,421011,Cease to Flow,Observed,0.277778,0.759528,0.533352,0.601597,Maintained
0,Macquarie-Castlereagh,421012,Cease to Flow,Observed,0.611111,0.025748,1.835605,0.087165,Degraded
